# Database-style joins

Say we have two datasets.  One is a list of students in a course, plus all the assignments they've submitted and their grades on those assignments.  The dataset has one row for each assignment, and looks something like this:

| Student ID | Assignment | Grade |
| --- | --- | --- |
| 001 | Midterm | 90 |
| 001 | Final | 78 |
| 002 | Midterm | 100 |
| 002 | Final | 100 |

Our second dataset is a list of students and their majors:

| Student ID | Major |
| --- | --- |
| 001 |  Physics |
| 002 |  History |

If we want to do some analysis of how grades differ by major, we'll need to combine these datasets somehow.  We want to get a dataset that looks something like this:

| Student ID | Assignment | Grade | Major |
| --- | --- | --- | --- |
| 001 | Midterm | 90 | Physics |
| 001 | Final | 78 | Physics |
| 002 | Midterm | 100 | History |
| 002 | Final | 100 | History |

But, there's an important question we need to answer.  If we have a student in one dataset but not the other, how do we want to handle that?  E.g., if we have a student who hasn't declared a major yet, and isn't in the second dataset?  We really only have two sensible options:

1. Don't include them in the final dataset.  We don't know their major, so it could throw things off.  If we want to do this, we'll use an *inner join:* combine two datasets, and only keep observation that have a corresponding entry in *both*.
2. Include them, but set their major to a missing value.  We can then analyze "no/missing major" students to see if there's any patterns with them.  This would be a *left join:* keep *all* observation from one dataset (we get to pick which one); if someone doesn't appear in the other one, just use missing values.

Note that there's also a *right join*, which is the same thing as a left join, but you just switch which dataset is the "keep everything from here" dataset, and an *outer join* (sometimes called a *full join*), which says "keep all observations from both datasets."  Right joins are pretty rare, since most people just use left joins; outer joins are uncommon, since they're not as widely useful.

When we join two datasets, we need to specify how to determine if two records "match."  We don't have a lot of flexibility here; we provide a column (or list of columns), and rows are combined if they have the same values from those columns.  This can sometimes cause issues: imagine joining two datasets based on first name.  Let's say it's a dataset about US Presidents.  If you do that, then "James Madison" and "James Monroe" will both match, and you'll get a messed up dataset.  In this case, you'd want to probably specify a join on first, middle, and last name, so that all three have to be identical for two records to be considered a match.

Pandas lets you do all of these kinds of joins using the `pd.merge()` function or the `DataFrame.merge()` method.  Let's make a small dataset and show what each one of these looks like.

In [1]:
import pandas as pd

assignments = pd.DataFrame(
    {
        "Student ID": [1, 1, 2, 2, 3, 3],
        "Assignment": ["Midterm", "Final", "Midterm", "Final", "Midterm", "Final"],
        "Grade": [90, 78, 100, 100, 50, 87],
    }
)

students = pd.DataFrame(
    {
        "Student ID": [1, 2, 4],
        "Major": ["Physics", "History", "English"],
    }
)

print(assignments)
print()
print(students)

   Student ID Assignment  Grade
0           1    Midterm     90
1           1      Final     78
2           2    Midterm    100
3           2      Final    100
4           3    Midterm     50
5           3      Final     87

   Student ID    Major
0           1  Physics
1           2  History
2           4  English


In order to join/merge these two datasets, we need to specify the *column(s) to merge on.*  Pandas will find all records from the two datasets where all the specified columns have *the same value*, and combine their data into a new record.

In [2]:
# Left join: keep all records from the `left=` dataset
print(
    pd.merge(
        left=assignments, # The "left" dataset
        right=students,   # The "right" dataset
        on="Student ID",  # The column where values need to match
        how="left",       # The kind of join (left/right/inner/outer)
    )
)

   Student ID Assignment  Grade    Major
0           1    Midterm     90  Physics
1           1      Final     78  Physics
2           2    Midterm    100  History
3           2      Final    100  History
4           3    Midterm     50      NaN
5           3      Final     87      NaN


In [3]:
# Right join: keep all records from the `right=` dataset
print(
    pd.merge(
        left=assignments,
        right=students,
        on="Student ID",
        how="right",
    )
)

   Student ID Assignment  Grade    Major
0           1    Midterm   90.0  Physics
1           1      Final   78.0  Physics
2           2    Midterm  100.0  History
3           2      Final  100.0  History
4           4        NaN    NaN  English


Note that `pd.merge(left=x, right=y, how="right")` is identical to `pd.merge(left=y, right=x, how="left")`--this is one of the reasons that right joins are pretty uncommon.

In [4]:
# Inner join: keep only records common to both datasets
print(
    pd.merge(
        left=assignments,
        right=students,
        on="Student ID",
        how="inner",
    )
)

   Student ID Assignment  Grade    Major
0           1    Midterm     90  Physics
1           1      Final     78  Physics
2           2    Midterm    100  History
3           2      Final    100  History


In [5]:
# Outer join: keep all records from both datasets.
print(
    pd.merge(
        left=assignments,
        right=students,
        on="Student ID",
        how="outer",
    )
)

   Student ID Assignment  Grade    Major
0           1    Midterm   90.0  Physics
1           1      Final   78.0  Physics
2           2    Midterm  100.0  History
3           2      Final  100.0  History
4           3    Midterm   50.0      NaN
5           3      Final   87.0      NaN
6           4        NaN    NaN  English


Lastly, an example of a join going wrong and needing more than one join column specified.

In [6]:
innaugurations = pd.DataFrame(
    {
        "First Name": ["James", "James"],
        "Last Name": ["Madison", "Monroe"],
        "Innaugurated": ["March 4, 1809", "March 4, 1817"]
    }
)
birthdays = pd.DataFrame(
    {
        "First Name": ["James", "James"],
        "Last Name": ["Madison", "Monroe"],
        "Birthday": ["March 16, 1751", "April 28, 1758"]
    }
)

print(
    pd.merge(
        left=innaugurations,
        right=birthdays,
        on="First Name",
        how="inner"
    )
)

  First Name Last Name_x   Innaugurated Last Name_y        Birthday
0      James     Madison  March 4, 1809     Madison  March 16, 1751
1      James     Madison  March 4, 1809      Monroe  April 28, 1758
2      James      Monroe  March 4, 1817     Madison  March 16, 1751
3      James      Monroe  March 4, 1817      Monroe  April 28, 1758


Uh-oh.  Notice how we now have two entries for each president!  That's not right.  Also notice how the "Last Name" column copied over, but one version has a "\_y" at the end of its name.  Pandas automatically adds suffixes like this whenever there are column with the same name in both dataset that are *not* part of the `on=` argument.  You can always drop this column later (or before the join), and you usually should.

Let's fix this join, by specifying multiple columns for the `on=` argument.

In [7]:
print(
    pd.merge(
        left=innaugurations,
        right=birthdays,
        on=["First Name", "Last Name"],
        how="inner"
    )
)

  First Name Last Name   Innaugurated        Birthday
0      James   Madison  March 4, 1809  March 16, 1751
1      James    Monroe  March 4, 1817  April 28, 1758


But there's an important question you migth have: what if the column names I need to join on are different across the two datasets?  E.g. one has "First Name" and the other just has "First"?  There are two solution:

1. Rename the columns in one of your datasets.
2. Instead of using `on=`, you can use `left_on=` and `right_on=`.  Give each argument a column or list of columns, and they'll be matched by position.

E.g.:

In [8]:
birthdays = pd.DataFrame(
    {
        "firstname": ["James", "James"],
        "lastname": ["Madison", "Monroe"],
        "Birthday": ["March 16, 1751", "April 28, 1758"]
    }
)

print(
    pd.merge(
        left=innaugurations,
        right=birthdays,
        # Joins where the "First Name" column from `innaugurations` equals
        # the "firstname" column from `birthdays`, AND where "Last Name"
        # from `innaugurations` equals "lastname" from `birthdays`.
        left_on=["First Name", "Last Name"],
        right_on=["firstname", "lastname"],
        how="inner"
    )
)

  First Name Last Name   Innaugurated firstname lastname        Birthday
0      James   Madison  March 4, 1809     James  Madison  March 16, 1751
1      James    Monroe  March 4, 1817     James   Monroe  April 28, 1758
